# 機械学習スクラッチ

NumPy などに備わっている基本的なライブラリを組み合わせることで、scikit-learn などの応用的なライブラリに実装されている機能と等価なクラス・関数を自作することができます。これをスクラッチと呼びます。


スクラッチを通して、scikit-learnなどのライブラリを動かすだけでは掴みづらい、アルゴリズムの深い理解を目指します。コーディングのスキル向上も兼ねていますが、それは主な目的ではありません。


以下のような効果を狙っています。


新たな手法に出会った時に理論・数式を理解しやすくする
ライブラリを使う上での曖昧さを減らす
既存の実装を読みやすくする

今回はまず、機械学習のプログラムを完全にはスクラッチせず、scikit-learn を用いて実装します。そして、次回から段階的に scikit-learn を用いた実装をスクラッチに移行していきます。


## 【問題1】train_test_split のスクラッチ
まずは、scikit-learnの train_test_split をスクラッチしてみます。以下の雛形をベースに関数を実装してください。


https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


なお、作成した関数がscikit-learnの train_test_split と同じ動作をするか必ず確認をしましょう。

scikit-learnを使ったコードを実装しベースラインモデルを作成していきます。


検証データの分割には問題1で作成した自作の関数を用いてください。クロスバリデーションではなくホールドアウト法で構いません。

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
%matplotlib inline

In [57]:
def scratch_train_test_split(X, y, train_size=0.8):
    """検証データを分割する。
    Parameters
    ----------
    X : ndarray
      訓練データ (n_samples, n_features)
    y : ndarray
      正解値 (n_samples,)
    train_size : float
      何割をtrainとするか指定 (0 < train_size < 1)
    Returns
    -------
    X_train : ndarray
      訓練データ (n_samples, n_features)
    X_test : ndarray
      検証データ (n_samples, n_features)
    y_train : ndarray
      訓練データの正解値 (n_samples,)
    y_test : ndarray
      検証データの正解値 (n_samples,)
    """
    # Error if feature samples number does not corresponds to y number.
    if X.shape[0] != y.shape[0]:
        raise ValueError("X samples number({}) is not same as y {}.".format(
                X.shape[0], y.shape[0]))

    # make several parameters to be used
    n_samples = X.shape[0]
    test_size = (1 - train_size)
    n_train = np.floor((1-test_size) * n_samples).astype(int)
    n_test = n_samples - n_train
    classes = np.unique(y)
    n_classes = len(classes)
    class_counts = np.bincount(y)
    class_indices = np.split(np.argsort(y, kind='mergesort'),
                             np.cumsum(class_counts)[:-1])
    X_test = X[:n_test]
    X_train = X[n_test:(n_test + n_train)]
    y_test = y[:n_test]
    y_train = y[n_test:(n_test + n_train)]

    return X_train, X_test, y_train, y_test

In [58]:
X = np.arange(10).reshape((5, 2))
y = np.array([1,1,0,1,0])
scratch_train_test_split(X, y)

(array([[2, 3],
        [4, 5],
        [6, 7],
        [8, 9]]),
 array([[0, 1]]),
 array([1, 0, 1, 0]),
 array([1]))

分類問題
分類は3種類の手法をscikit-learnを使って実装します。

ロジスティック回帰
SVM
決定木

scikit-learnにおいてLogisticRegressionクラスとSGDClassifierクラスの2種類から使用できます。ここでは勾配降下法を用いて計算するSGDClassifierクラスを利用してください。引数でloss=”log”とすることでロジスティック回帰の計算になります。

scikit-learn にはロジスティック回帰に使える分類器として LogisticRegression クラスと SGDClassifier クラスが用意されています。ここでは勾配降下法を用いて計算する SGDClassifier クラスを用います。引数で loss="log" を指定することでロジスティック回帰の計算ができます。

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

3種類のデータセットを用いて動作を確認します。

1つ目は事前学習期間同様のirisデータセットです。 https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html 2値分類としたいため、以下の2つの目的変数のみ利用します。特徴量は4種類すべて使います。

virgicolorとvirginica

残り2つは特徴量が2つのデータセットを人工的に用意します。以下のコードで説明変数X,目的変数yが作成可能です。「シンプルデータセット1」「シンプルデータセット2」とします。特徴量が2つであるため可視化が容易です。

シンプルデータセット1作成コード

## 【問題2】 分類問題を解くコードの作成
上記3種類の手法で3種類のデータセットを学習・推定するコードを作成してください。

##### ロジスティック回帰

In [59]:
# アイリスデータを抽出
iris_dataset = load_iris()
#display(iris_dataset)
display(iris_dataset["target_names"])
display(iris_dataset["feature_names"])

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [60]:
X = iris_dataset.data[50:]
y = iris_dataset.target[50:]
display(X.shape)
display(y.shape)

(100, 4)

(100,)

In [61]:
X_train, X_test, y_train, y_test = scratch_train_test_split(X, y)
X_train.shape

(80, 4)

In [62]:
rog = SGDClassifier(loss="log")
rog.fit(X_train, y_train)

SGDClassifier(loss='log')

In [63]:
y_pred = rog.predict(X_test)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1])

In [64]:
accuracy_score(y_test, y_pred)

0.95

##### SVM

In [65]:
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X1 = np.concatenate([f0, f1])
y1 = np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])
#random_index = np.random.permutation(np.arange(n_samples))
#X = X[random]

In [66]:
X_train1, X_test1, y_train1, y_test1 = scratch_train_test_split(X, y)
X_train1.shape

(80, 4)

In [67]:
svm = SVC()
svm.fit(X_train1, y_train1)
y_pred1 = rog.predict(X_test1)
y_pred1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1])

##### 決定木

In [68]:
X = np.array([
    [-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
    [ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
    [-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
    [ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
    [-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
    [-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
    [-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
    [-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
    [ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
    [-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
    [-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
    [ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
    [ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
    [-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
    [ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
    [-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
    [ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
    [-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
    [ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
    [-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ],
])
y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [69]:
X_train, X_test, y_train, y_test = scratch_train_test_split(X, y)
print(X.shape,X_train.shape)

(40, 2) (32, 2)


In [70]:
tree =DecisionTreeClassifier()
tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [71]:
y_pred2 = tree.predict(X_test)
y_pred2

array([0, 0, 0, 0, 0, 1, 0, 1])

##### 回帰問題
次に回帰は1種類をscikit-learnを使って実装します。


線形回帰

線形回帰は勾配降下法を用いて計算する SGDRegressor クラスを利用してください。


https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html


データセットは事前学習期間同様にHouse Pricesコンペティションのものを使います。


https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data


train.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。

## 【問題3】 回帰問題を解くコードの作成
線形回帰でHouse Pricesデータセットを学習・推定するコードを作成してください。

In [72]:
df = pd.read_csv("train.csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [76]:
X = df.loc[:,["GrLivArea","YearBuilt"]].values
y = df.loc[:,["SalePrice"]].values
X_train, X_test, y_train, y_test = train_test_split(X, y)
reg = SGDRegressor()
reg.fit(X_train, y_train.ravel())
y_pred = reg.predict(X_test)
y_pred

array([-5.21660024e+15, -5.55528114e+15, -5.03721547e+15, -4.20296408e+15,
       -5.00491126e+15, -4.60376602e+15, -5.09447408e+15, -4.89202315e+15,
       -5.69314849e+15, -5.22173839e+15, -5.22044433e+15, -5.45477694e+15,
       -5.30507213e+15, -4.43053609e+15, -5.04388695e+15, -5.02346327e+15,
       -4.51300822e+15, -4.89331556e+15, -4.49027379e+15, -5.23121240e+15,
       -5.02866902e+15, -4.57043186e+15, -4.55257477e+15, -5.10320876e+15,
       -5.06276901e+15, -4.46395442e+15, -4.61564324e+15, -4.18808285e+15,
       -4.61324996e+15, -5.59510351e+15, -4.63590023e+15, -4.38934376e+15,
       -4.61521575e+15, -4.51258405e+15, -5.14429869e+15, -4.62974704e+15,
       -4.74152766e+15, -4.90108127e+15, -5.30069319e+15, -5.97523422e+15,
       -4.86475635e+15, -4.81639249e+15, -4.56137871e+15, -4.51659316e+15,
       -4.67188514e+15, -5.27120809e+15, -4.34609485e+15, -4.51197496e+15,
       -5.30968205e+15, -5.17652370e+15, -4.75340654e+15, -5.26095168e+15,
       -4.27440569e+15, -